In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.generation.utils import GenerationConfig
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate

from prompt_template import get_template

import warnings
warnings.filterwarnings("ignore")

C:\Users\91591\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin C:\Users\91591\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


In [2]:
# run this to download model weights
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="baichuan-inc/Baichuan2-7B-Chat-4bits", local_dir="./model_weights/baichuan-inc/Baichuan2-7B-Chat-4bits")
#snapshot_download(repo_id="baichuan-inc/Baichuan2-13B-Chat-4bits", local_dir="./model_weights/baichuan-inc/Baichuan2-13B-Chat-4bits")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan2-7B-Chat-4bits", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./model_weights/baichuan-inc/Baichuan2-7B-Chat-4bits", load_in_4bit=True, 
                                             trust_remote_code=True)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
model = model.to('cuda')
tokenizer.bos_token = '[INST]'
tokenizer.eos_token = '[/INST]'
tokenizer.pad_token = tokenizer.eos_token

In [6]:
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, device=0
)
hf = HuggingFacePipeline(pipeline=pipe)
prompt = PromptTemplate(template=get_template(), input_variables=['instruction','history','user'])
llm_chain = LLMChain(prompt=prompt, llm=hf)

The model 'BaichuanForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Reform

In [7]:
llm_chain.run(instruction='你是一个热情可爱美少女',history='',user='你好')

'你好，很高兴见到你！有什么我可以帮助你的吗？'

In [8]:
prompt

PromptTemplate(input_variables=['instruction', 'history', 'user'], output_parser=None, partial_variables={}, template='[INST] {instruction}，对<>内的内容进行回答\n\n{history}\n\n<{user}>\n', template_format='f-string', validate_template=True)

In [10]:
model.generation_config = GenerationConfig.from_pretrained("baichuan-inc/Baichuan2-13B-Chat-4bits")
messages = []

messages.append({"role": "user", "content": "解释一下“温故而知新”"})
response = model.chat(tokenizer, messages)
print(response)

"温故而知新"是中国古代儒家思想中的一种教育理念，出自《论语·为政》。这句话的意思是：通过回顾和复习已有的知识，能够发现新的知识和见解。这种教育理念强调在学习过程中，不仅要学习新知识，还要对已有知识进行巩固和梳理，以便更好地理解和掌握新知识。

"温故而知新"这一理念在现代教育中仍然具有重要的指导意义。在学习过程中，我们需要不断地回顾和复习过去的知识，以便更好地理解和应用新知识。同时，这一理念也强调了实践和体验的重要性，认为通过实践和经验积累，我们能够发现新的知识和见解。


In [ ]:
messages.append({"role": "assisstant", "content":response})
messages.append({"role": "user", "content": "请介绍一下自己"})
response = model.chat(tokenizer, messages)
print(response)